# DATA MINING PROJECT: Analysis of a Supermarket’s Customers
## 4) Pattern Mining
### *Antonio Strippoli, Valerio Mariani*

In [1]:
from functions import *  # Custom function for the analysis
from gsp import apriori
import datetime
import logging
import time
import os

# Set logging
if os.path.exists('log.txt'):
    os.remove('log.txt')
logging.basicConfig(level=logging.INFO, filename="log.txt", filemode="a+", format="%(message)s")
logging.getLogger().addHandler(logging.StreamHandler())

Temporary cell here

In [7]:
# Config (which result do we want to analyze)
min_baskets = 10
min_sup = 0.25
max_gap = datetime.timedelta(days=365)
min_gap = datetime.timedelta(days=365)

# Read the dataset
df = read_dataset()
# Remove some baskets
df = remove_baskets(df, min_baskets)
# Convert into seq form
seq_data, time_stamps = sequentialize(df, return_times=True)

# Apply GSP
result_set = apriori(seq_data, min_sup, time_stamps, max_span=None, min_gap=None, max_gap=max_gap)
print_distribution(result_set)

# Distribution of lengths: {1: 56, 2: 1, 3: 0, 4: 0, 5: 0}

Distribution of lengths: {1: 56, 2: 13, 3: 1, 4: 0, 5: 0}
Sequences containing duplicates: 9 / 70


### Apply GSP on sequential data

In [ ]:
# Main cycle: apply GSP multiple times
params = {
    'min_sup': [0.4, 0.35, 0.3, 0.25, 0.2, 0.15],
    'min_baskets': [20, 10, 5, 3, 2],
}
for min_sup in params['min_sup']:
    for min_baskets in params['min_baskets']:
        logging.info(f"MIN_BASKETS: {min_baskets}, MIN_SUP: {min_sup}")

        # Read the dataset
        df = read_dataset()
        # Remove some baskets
        df = remove_baskets(df, min_baskets)
        # Convert into seq form
        seq_data = sequentialize(df)
        
        # Apply GSP
        t0 = time.time()
        result_set = apriori(seq_data, min_sup, verbose=False)
        t1 = time.time()

        # Compute n. of sequences with len > 2 and n. of sequences containing duplicates
        cnt_len_2 = 0
        cnt_duplicates = 0
        for r in result_set:
            r = r[0]
            tmp = []
            for l in r:
                tmp.extend(l)
            if len(tmp) >= 2:
                cnt_len_2 += 1
                if len(set(tmp)) < len(tmp):
                    cnt_duplicates += 1

        logging.info(
            f"TOTAL TIME:\t{round(t1-t0, 2)} s\n"\
            f"LEN RESULT SET:\t{len(result_set)}\n"\
            f"LEN SEQ > 2:\t{cnt_len_2}\nN. DUPLICATES:\t{cnt_duplicates}\n"
        )

        # Save
        save_to_pickle(result_set, min_baskets, min_sup)

### Considerazioni

Prendendo quelli che hanno fatto almeno 20 baskets, otteniamo una mole maggiore di risultati. Abbassando min_baskets, il supporto comincia a diventare sempre più basso in generale, il che lascia comunque presagire che nei clienti più occasionali non ci siano pattern evidenti.

Il parametro min_baskets è importante perché altrimenti trovare dei pattern un po' più interessanti (che spazino tra basket diversi) è molto difficile (richiedono min_support bassi che alzano il costo computazionale). Alla fine abbiamo scelto 10 come giusto compromesso tra un numero di clienti abbastanza alto (il 10% di quelli di partenza) e sequenze variegate/lunghe.

### Analyze results and collect statistics

In [ ]:
# Config (which result do we want to analyze)
min_baskets = 10
min_sup = 25

# Read result
result_set = read_result(min_baskets, min_sup)
result_set = convert_tuples_to_list(result_set)
print_distribution(result_set)

# Read and prepare the dataset
df = read_dataset()
df = remove_baskets(df, min_baskets)

# Compute mean qta values
result_set = compute_patterns_mean_qta(result_set, df)

# Convert ProdID to ProdDescr
result_set = prodID_to_prodDescr(result_set, df)

result_set

### Considerazioni su RESULT_SET
- Conta sequenze che hanno doppioni e quelle che non ce l'hanno (in percentuale?)
- ogni evento è formato da un singolo elemento (non ci sono carrelli che hanno in comune più di un elemento, ma solo sequenze di carrelli con oggetti comuni)
- Contare quante sono sequenze lunghe 1, 2, 3...

### Considerazioni su transazioni e clienti a cui si riferiscono le sequenze
- quantità media di oggetti presa per ogni oggetto di ogni sequenza
- tempo passato tra una transazione e l'altra

### Considerazioni
due casi:
- oggetti uguali: l'oggetto vende bene? Vengono ricomprati per essere venduti ancora
- oggetti diversi: oggetto1 magari non ha venduto benissimo e se n'è comprato uno simile per provare a vendere quello, oppure ha comprato semplicemente un'altra variante.

### Apply Time Constraints

In [ ]:
params = {
    'min_sup': [0.4, 0.35, 0.3, 0.25, 0.2, 0.15],
    'min_baskets': [20, 10],
    'max_gap': [datetime.timedelta(days=1), datetime.timedelta(days=2), datetime.timedelta(days=3), datetime.timedelta(weeks=1), datetime.timedelta(weeks=2), datetime.timedelta(weeks=3), datetime.timedelta(weeks=4), datetime.timedelta(weeks=8), datetime.timedelta(weeks=12)],
    'max_span': [datetime.timedelta(weeks=4), datetime.timedelta(weeks=8), datetime.timedelta(weeks=12), datetime.timedelta(weeks=48)]
}
for min_sup in params['min_sup']:
    for min_baskets in params['min_baskets']:
        for max_gap in params['max_gap']:
            for max_span in params['max_span']:
                logging.info(f"MIN_BASKETS: {min_baskets}, MIN_SUP: {min_sup}, MAX_GAP: {max_gap}, MAX_SPAN: {max_span}")

                # Read the dataset
                df = read_dataset()
                # Remove some baskets
                df = remove_baskets(df, min_baskets)
                # Convert into seq form
                seq_data, time_stamps = sequentialize(df, return_times=True)
                
                # Apply GSP
                t0 = time.time()
                result_set = apriori(seq_data, min_sup, time_stamps, max_span=max_span, min_gap=None, max_gap=max_gap)
                t1 = time.time()

                # Compute n. of sequences with len > 2 and n. of sequences containing duplicates
                cnt_len_2 = 0
                cnt_duplicates = 0
                for r in result_set:
                    r = r[0]
                    tmp = []
                    for l in r:
                        tmp.extend(l)
                    if len(tmp) >= 2:
                        cnt_len_2 += 1
                        if len(set(tmp)) < len(tmp):
                            cnt_duplicates += 1

                logging.info(
                    f"TOTAL TIME:\t{round(t1-t0, 2)} s\n"\
                    f"LEN RESULT SET:\t{len(result_set)}\n"\
                    f"LEN SEQ > 2:\t{cnt_len_2}\nN. DUPLICATES:\t{cnt_duplicates}\n"
                )

                # Save
                save_to_pickle(result_set, min_baskets, min_sup, max_gap.days, max_span.days)

#### Re-apply the procedure introducing time constraints

In [ ]:
# Config (which result do we want to analyze)
min_baskets = 10
min_sup = 0.25
max_gap = 24
max_span = None

cnt = 0
while True:
    print("TESTING max_gap =", max_gap)
    # Read the dataset
    df = read_dataset()
    # Remove some baskets
    df = remove_baskets(df, min_baskets)
    # Convert into seq form
    seq_data, time_stamps = sequentialize(df, return_times=True)

    # Apply GSP
    result_set = apriori(seq_data, min_sup, time_stamps, max_span=None, min_gap=None, max_gap=datetime.timedelta(weeks=max_gap))
    max_gap += 4

    if not cnt:
        cnt = len(result_set)
        print('LEN RESULT SET:', cnt)
        continue

    if cnt > len(result_set):
        cnt = len(result_set)
        print('LEN RESULT SET INCREMENTED:', cnt)
        if cnt >= 73:
            assert False

# Sort by support
result_set.sort(key=lambda x: x[1], reverse=True)
# Prepare a copy
result_set_original = result_set

# Read and prepare the dataset
df = read_dataset()
df = remove_baskets(df, min_baskets)